# **Pivot, Melt, Stack & Unstack**

## **Pendahuluan**

Kotak masuk emailku tak hentinya menerima sejumlah link baru dari Andra untuk bab-bab yang akan kupelajari di modul Pandas part 2 ini. Banyak sekali referensi dari Andra!

“Pivot, Melt, Stack, dan Unstack, apa ini?” gumamku sendiri membaca subject email Andra.

Aku pun bergegas mengaksesnya:

Reference: https://pandas.pydata.org/pandas-docs/stable/user_guide/reshaping.html

 

Melakukan format ulang pada dataset itu sangatlah penting, biasanya hal ini dilakukan untuk mengetahui keseluruhan data secara cepat dengan chart atau visualisasi. Untuk orang yang sudah mahir menggunakan spreadsheet pastilah tau banyak tentang fitur pivot ini.

Di Pandas, ada beberapa teknik untuk melakukan pivot atau unpivot yang biasa disebut **melt**. Terdapat pula konsep **stack** yang artinya menumpuk data dengan kolom yang lebih sedikit (stack) sama seperti konsep melt dan ada pula yang memperluas data dengan kolom yang lebih banyak (unstack) sama seperti konsep **pivot**.

Untuk memahami konsep pivot, melt, stack, dan unstack pada Pandas mari persiapkan dataset sederhana terlebih dahulu.

## **Membuat Data dan Mencari Nilai Unik dari Tiap Kolom**

In [1]:
import pandas as pd
# Membuat Dataframe
data = pd.DataFrame({
  'kelas': 6*['A'] + 6*['B'],
  'murid': 2*['A1'] + 2*['A2'] + 2*['A3'] + 2*['B1'] + 2*['B2'] + 2*['B3'],
  'pelajaran': 6*['math','english'],
  'nilai': [90,60,70,85,50,60,100,40,95,80,60,45]
}, columns=['kelas','murid','pelajaran','nilai'])

print("Data:\n", data)
print(f'\nData terdiri dari {data.shape[0]} baris dan {data.shape[1]} kolom.')

Data:
    kelas murid pelajaran  nilai
0      A    A1      math     90
1      A    A1   english     60
2      A    A2      math     70
3      A    A2   english     85
4      A    A3      math     50
5      A    A3   english     60
6      B    B1      math    100
7      B    B1   english     40
8      B    B2      math     95
9      B    B2   english     80
10     B    B3      math     60
11     B    B3   english     45

Data terdiri dari 12 baris dan 4 kolom.


In [2]:
# Unique value pada setiap kolom data
for column in data.columns:
    print('Unique value %s: %s' % (column, data[column].unique()))

Unique value kelas: ['A' 'B']
Unique value murid: ['A1' 'A2' 'A3' 'B1' 'B2' 'B3']
Unique value pelajaran: ['math' 'english']
Unique value nilai: [ 90  60  70  85  50 100  40  95  80  45]


## **Pivot**

Untuk menerapkan menerapkan method `.pivot()` pada dataframe dapat dilakukan pada dataframe yang miliki index tunggal ataupun indexnya adalah multiindex.

Untuk dataset yang masih sama, yaitu data.

In [3]:
# Pivoting with single column measurement
pivot1 = data.pivot(index='murid',columns='pelajaran',values='nilai')
print('Pivoting with single column measurement:\n', pivot1)

# Pivoting with multiple column measurement
pivot2 = data.pivot(index='murid',columns='pelajaran')
print('\n\nPivoting with multiple column measurement:\n', pivot2)

Pivoting with single column measurement:
 pelajaran  english  math
murid                   
A1              60    90
A2              85    70
A3              60    50
B1              40   100
B2              80    95
B3              45    60


Pivoting with multiple column measurement:
             kelas        nilai     
pelajaran english math english math
murid                              
A1              A    A      60   90
A2              A    A      85   70
A3              A    A      60   50
B1              B    B      40  100
B2              B    B      80   95
B3              B    B      45   60


**Penjelasan:**

Apa yang berbeda dari kedua code di atas? Pada code pertama di-specify values mana yang akan dilakukan pivot sedangkan pada yang kedua tidak spesifik mana yang akan dilakukan pivot, untuk itu Pandas secara default men-treat kolom yang ada selain yang telah di-specify sebagai index dan columns sebagai values.

## **Pivot_table**

Apa yang terjadi kalau output pivot tabel memiliki duplicate index? Seperti yang diketahui, index di dataframe adalah unique identifier untuk setiap row, jadi tidak boleh ada duplikat dan setiap membuat pivot tabel, harus specify index as kolom yang mana dan columns nya memakai kolom yang mana.

Perhatikan contoh yang diilustrasikan berikut ini:



```
pivot_tab = data.pivot(index='kelas', columns='pelajaran',values='nilai')
```


jika kita menjalankan program diatas maka kita akan mendapatkan output error berupa:

<font color='red'>`ValueError: Index contains duplicate entries, cannot reshape`</font>


Hal ini dapat diatasi dengan melakukan method `.pivot_table()` pada dataframe. Metode ini sama seperti melakukan pivot pada tabel tapi juga melakukan **groupby** dan **aggregation** (`aggfunc`) pada level rows sehingga dipastikan tidak ada duplicate index di rows (secara default `aggfunc = 'mean'`).


Keyword aggfunc yang digunakan pada method `.pivot_table()` dapat menggunakan nilai berikut:

`'sum'`,
`'mean'`,
`'median'`




In [4]:
# Creating pivot and assign pivot_tab dengan menggunakan keyword aggfunc='sum'
pivot_tab_mean = data.pivot_table(index='kelas',columns='pelajaran',values='nilai',aggfunc='sum')
print('Creating pivot table -- aggfunc mean:\n', pivot_tab_mean)

# Creating pivot and assign pivot_tab dengan menggunakan keyword aggfunc='mean'
pivot_tab_mean = data.pivot_table(index='kelas',columns='pelajaran',values='nilai',aggfunc='mean')
print('\n\nCreating pivot table -- aggfunc mean:\n', pivot_tab_mean)

# Creating pivot and assign pivot_tab dengan menggunakan keyword aggfunc='median'
pivot_tab_median = data.pivot_table(index='kelas',columns='pelajaran',values='nilai',aggfunc='median')
print('\n\nCreating pivot table -- aggfunc median:\n', pivot_tab_median)

Creating pivot table -- aggfunc mean:
 pelajaran  english  math
kelas                   
A              205   210
B              165   255


Creating pivot table -- aggfunc mean:
 pelajaran    english  math
kelas                     
A          68.333333  70.0
B          55.000000  85.0


Creating pivot table -- aggfunc median:
 pelajaran  english  math
kelas                   
A               60    70
B               45    95


## **Melt - Part 1**

Teknik melt melalui `pd.melt()` digunakan untuk mengembalikan kondisi data yang sudah dilakukan pivot menjadi sebelum pivot.

Mari diperhatikan kembali dataframe yang telah digunakan sebelumnya dan dataframenya sudah dipivot.



In [5]:
# Mempivot data
pivot3 = data.pivot(index='murid',columns='pelajaran',values='nilai')
print('Pivoting with single column measurement:\n', pivot3)

Pivoting with single column measurement:
 pelajaran  english  math
murid                   
A1              60    90
A2              85    70
A3              60    50
B1              40   100
B2              80    95
B3              45    60


Akan melakukan teknik melting pada dataframe output di atas.

In [6]:
melt_piv_1 = pd.melt(pivot3)
melt_piv_1

,pelajaran,value
0,english,60
1,english,85
2,english,60
3,english,40
4,english,80
5,english,45
6,math,90
7,math,70
8,math,50
9,math,100


Dapat kita lihat bahwa output diatas tidak mirip dengan data yang kita miliki. Untuk mendapatkan hasil yang mirip dengan data original yang kita punya di awal, kita perlu me-reset index pada pivot table terlebih dahulu sebelum melakukan melting, dan selanjutnya menjadikan kolom 'murid' sebagai kolom id dari tiap data. Selanjutnya, kita akan urutkan berdasarkan kolom murid dan mereset index-nya agar lebih mudah dibaca. Sebelum kita contohkan, mari lihat contoh lainnya.

contoh lainnya adalah:

In [14]:
# Pivoting dataframe
data_pivot = data.pivot_table(index='kelas',columns='pelajaran',values='nilai',\
                              aggfunc='mean').reset_index()
print('Pivoting dataframe:\n', data_pivot)

# [1] Melting dataframe data_pivot
data_melt_1 = pd.melt(data_pivot)
print('\n\nMelting dataframe:\n', data_melt_1)



Pivoting dataframe:
 pelajaran kelas    english  math
0             A  68.333333  70.0
1             B  55.000000  85.0


Melting dataframe:
   pelajaran    value
0     kelas        A
1     kelas        B
2   english  68.3333
3   english       55
4      math       70
5      math       85


kita juga menspesifikkan keyword argument `id_vars` yang ditujukan untuk membuat fix kolom yang akan berfungsi sebagai id tiap barisnya.

In [ ]:
# [2] Melting dataframe data_pivot dengan id_vars
data_melt_2 = pd.melt(data_pivot,id_vars='kelas')
print('Melting dataframe dengan idvars:\n', data_melt_2)

Melting dataframe dengan idvars:
   kelas pelajaran      value
0     A   english  68.333333
1     B   english  55.000000
2     A      math  70.000000
3     B      math  85.000000


Berikut adalah contoh perbaikan dari **pivot 3** diatas.

In [12]:
# Mempivot data
pivot4 = data.pivot(index='murid',columns='pelajaran',\
                    values='nilai').reset_index()
print('Pivoting with single column measurement:\n', pivot4)

Pivoting with single column measurement:
 pelajaran murid  english  math
0            A1       60    90
1            A2       85    70
2            A3       60    50
3            B1       40   100
4            B2       80    95
5            B3       45    60


In [17]:
melt_piv_2 = pd.melt(pivot4, id_vars='murid').sort_values('murid')
melt_piv_2

,murid,pelajaran,value
0,A1,english,60
6,A1,math,90
1,A2,english,85
7,A2,math,70
2,A3,english,60
8,A3,math,50
3,B1,english,40
9,B1,math,100
4,B2,english,80
10,B2,math,95


### **Melt - Part 2**

Mari melanjutkan ke bagian kedua dari penggunaan teknik melt ini. Mari lihat kembali dataframe yang telah diperoleh melalui pivoting.

In [ ]:
import pandas as pd
# Dataframe
data = pd.DataFrame({
  'kelas': 6*['A'] + 6*['B'],
  'murid': 2*['A1'] + 2*['A2'] + 2*['A3'] + 2*['B1'] + 2*['B2'] + 2*['B3'],
  'pelajaran': 6*['math','english'],
  'nilai': [90,60,70,85,50,60,100,40,95,80,60,45]
}, columns=['kelas','murid','pelajaran','nilai'])

# Pivoting dataframe
data_pivot = data.pivot_table(index='kelas',columns='pelajaran',values='nilai',\
                              aggfunc='mean').reset_index()
print('Pivoting dataframe:\n', data_pivot)

Pivoting dataframe:
 pelajaran kelas    english  math
0             A  68.333333  70.0
1             B  55.000000  85.0


Lanjutkan dengan melakukan teknik melting pada dataframe output di atas untuk keyword argumen lainnya, yaitu:

[3] Dengan menspesifikasikan keyword argument value_vars yang digunakan untuk menampilkan variasi value apa saja yang perlu dimunculkan di kolom variable. 

In [ ]:
# [3.a] Melting dataframe data_pivot dengan value_vars
data_melt_3a = pd.melt(data_pivot, value_vars=['math'])
print('Melting dataframe dengan value_vars:\n', data_melt_3a)


Melting dataframe dengan value_vars:
   pelajaran  value
0      math   70.0
1      math   85.0


In [ ]:
# [3.b] Melting dataframe data_pivot dengan id_vars dan value_vars
data_melt_3b = pd.melt(data_pivot, id_vars='kelas', value_vars=['math'])
print('Melting dataframe dengan id_vars dan value_vars:\n', data_melt_3b)

Melting dataframe dengan id_vars dan value_vars:
   kelas pelajaran  value
0     A      math   70.0
1     B      math   85.0


[4] Dengan spesifikasikan keyword argument var_name dan value_name yang digunakan untuk menampilkan nama kolom untuk variable dan value

In [ ]:
# [4] Melting dataframe data_pivot dengan id_vars, value_vars, var_name. dan value_name
data_melt_4 = pd.melt(data_pivot, id_vars='kelas',\
                      value_vars=['english','math'], var_name='matpel', \
                      value_name='nilai (avg)')
print('\nMelting dataframe dengan id_vars, value_vars, var_name. dan value_name:\n', data_melt_4)


Melting dataframe dengan id_vars, value_vars, var_name. dan value_name:
   kelas   matpel  nilai (avg)
0     A  english    68.333333
1     B  english    55.000000
2     A     math    70.000000
3     B     math    85.000000


### **Stack & Unstack - Part 1**

Konsep **stacking** dan **unstacking** sama dengan **melt** dan **pivot**, hanya saja tidak memasukkan index sebagai parameter di stack/unstack tapi harus set index terlebih dahulu, baru bisa melakukan stacking/unstacking dengan level yang bisa ditentukan sendiri dengan menggunakan level name maupun level position.

In [ ]:
# Dataframe
data = pd.DataFrame({
  'kelas': 6*['A'] + 6*['B'],
  'murid': 2*['A1'] + 2*['A2'] + 2*['A3'] + 2*['B1'] + 2*['B2'] + 2*['B3'],
  'pelajaran': 6*['math','english'],
  'nilai': [90,60,70,85,50,60,100,40,95,80,60,45]
}, columns=['kelas','murid','pelajaran','nilai'])
print('Dataframe:\n', data)

Dataframe:
    kelas murid pelajaran  nilai
0      A    A1      math     90
1      A    A1   english     60
2      A    A2      math     70
3      A    A2   english     85
4      A    A3      math     50
5      A    A3   english     60
6      B    B1      math    100
7      B    B1   english     40
8      B    B2      math     95
9      B    B2   english     80
10     B    B3      math     60
11     B    B3   english     45


In [ ]:
# Set index data untuk kolom kelas, murid, dan pelajaran
data = data.set_index(['kelas','murid','pelajaran'])
print('Dataframe multi index:\n', data)

Dataframe multi index:
                        nilai
kelas murid pelajaran       
A     A1    math          90
            english       60
      A2    math          70
            english       85
      A3    math          50
            english       60
B     B1    math         100
            english       40
      B2    math          95
            english       80
      B3    math          60
            english       45


In [ ]:
# [1] Unstacking dataframe
data_unstack_1 = data.unstack()
print('Unstacking dataframe:\n', data_unstack_1)

Unstacking dataframe:
               nilai     
pelajaran   english math
kelas murid             
A     A1         60   90
      A2         85   70
      A3         60   50
B     B1         40  100
      B2         80   95
      B3         45   60


In [ ]:
# [2] Unstacking dengan specify level name
data_unstack_2 = data.unstack(level='murid')
print('Unstacking dataframe dengan level name:\n', data_unstack_2)

Unstacking dataframe dengan level name:
                 nilai                               
murid              A1    A2    A3     B1    B2    B3
kelas pelajaran                                     
A     english    60.0  85.0  60.0    NaN   NaN   NaN
      math       90.0  70.0  50.0    NaN   NaN   NaN
B     english     NaN   NaN   NaN   40.0  80.0  45.0
      math        NaN   NaN   NaN  100.0  95.0  60.0


In [ ]:
# [3] Unstacking dengan specify level position
data_unstack_3 = data.unstack(level=1)
print('Unstacking dataframe dengan level position:\n', data_unstack_3)

Unstacking dataframe dengan level position:
                 nilai                               
murid              A1    A2    A3     B1    B2    B3
kelas pelajaran                                     
A     english    60.0  85.0  60.0    NaN   NaN   NaN
      math       90.0  70.0  50.0    NaN   NaN   NaN
B     english     NaN   NaN   NaN   40.0  80.0  45.0
      math        NaN   NaN   NaN  100.0  95.0  60.0


### **Stack & Unstack - Part 2**

Dalam bagian kedua dari Stack & Unstack ini akan membahas stacking dataframe. Untuk itu, mari diperhatikan dataframe berikut ini

In [ ]:
# Dataframe
data = pd.DataFrame({
  'kelas': 6*['A'] + 6*['B'],
  'murid': 2*['A1'] + 2*['A2'] + 2*['A3'] + 2*['B1'] + 2*['B2'] + 2*['B3'],
  'pelajaran': 6*['math','english'],
  'nilai': [90,60,70,85,50,60,100,40,95,80,60,45]
}, columns=['kelas','murid','pelajaran','nilai'])
data = data.set_index(['kelas','murid','pelajaran'])
data_unstack = data.unstack(level=1)
print('Dataframe:\n', data_unstack)

Dataframe:
                 nilai                               
murid              A1    A2    A3     B1    B2    B3
kelas pelajaran                                     
A     english    60.0  85.0  60.0    NaN   NaN   NaN
      math       90.0  70.0  50.0    NaN   NaN   NaN
B     english     NaN   NaN   NaN   40.0  80.0  45.0
      math        NaN   NaN   NaN  100.0  95.0  60.0


Selajutnya, dengan menggunakan dataframe `data_unstack`, kita akan melakukan:

[1] Stacking dataframe 

In [ ]:
# [1] Stacking dataframe
data_stack = data_unstack.stack()
print('Stacked dataframe:\n', data_stack)

Stacked dataframe:
                        nilai
kelas pelajaran murid       
A     english   A1      60.0
                A2      85.0
                A3      60.0
      math      A1      90.0
                A2      70.0
                A3      50.0
B     english   B1      40.0
                B2      80.0
                B3      45.0
      math      B1     100.0
                B2      95.0
                B3      60.0


[2] Tukar posisi index setelah stacking dataframe

In [ ]:
# [2] Tukar posisi index setelah stacking dataframe
data_swap = data_stack.swaplevel(1,2)
print('Swapped data:\n', data_swap)

Swapped data:
                        nilai
kelas murid pelajaran       
A     A1    english     60.0
      A2    english     85.0
      A3    english     60.0
      A1    math        90.0
      A2    math        70.0
      A3    math        50.0
B     B1    english     40.0
      B2    english     80.0
      B3    english     45.0
      B1    math       100.0
      B2    math        95.0
      B3    math        60.0


In [ ]:
# [3] Melakukan sort_index pada stacking dataframe
data_sort = data_swap.sort_index()
print('Sorted data:\n', data_sort)

Sorted data:
                        nilai
kelas murid pelajaran       
A     A1    english     60.0
            math        90.0
      A2    english     85.0
            math        70.0
      A3    english     60.0
            math        50.0
B     B1    english     40.0
            math       100.0
      B2    english     80.0
            math        95.0
      B3    english     45.0
            math        60.0
